In [1]:
//init cell

//common functions

const os = require('os');
const fs = require('fs');
const osc = require('osc');
const net = require('net');

function sliceStartStop(loop) {
    return loop.slice(1, -1)
}

function sortByTs(loop) {
    return loop.sort((e1, e2) => e1.ts-e2.ts);
}

const defaultGestureData = { //loopKey must ALWAYS be manually provided
    pos: "default",
    duration: 5,
    group: "default",
    key: "default",
    looping: true,
    deltaLoop: true,
    deltaAccumulate: true,
    rotation: 0
};

function sendConfig(sketchId, conf) {
    conf = Array.isArray(conf) ? conf : [conf];
    conf = conf.map(cnf => ({...defaultGestureData, ...cnf}));
    // udp_osc_port.send({address: "/launchConfig", args: oscArgs(JSON.stringify(conf))});
    const jsonStr = JSON.stringify({type: "launchConfig", data: conf, sketchId});
    sockets.forEach(sock => sock.write(jsonStr + '\n' ));
}

function sendOscToSketch(sketchId, address, ...args) {
    udp_osc_port.send({address: '/'+sketchId+address, args: oscArgs(...args)});
}   
function clearSketch(sketchId) {
    sendOscToSketch(sketchId, "/clearAll");
} 

function normalizeTime(loop) {
    const start_time = loop[0].ts;
    const dur = loop.slice(-1)[0].ts - start_time;
    touch_pts = loop.map(({ts, pos}) => ({pos, ts: (ts-start_time)/dur}));
    return touch_pts;
}

function deduplicate(loop) { //for fixing touchOSC bug of "lagging Y" redundant pre-message
    let filteredLoop = loop.filter((elem, i, arr) => i%2 != 0); //throw out even indices
    return filteredLoop;
    //todo - this only works when messages are guaranteed ordered (local UDP, tcp)
    //for non-ordered messages, probably most robust is to just toss first couple values to
    //get rid of "cut start" from redundant message (or to move cursor near intened start before recording)
}

function flipY(loop) {
    return loop.map(({ts, pos: {x, y}}) => ({ts, pos: {x, y: 1-y}}))
}

function getDelta(loop) {
    let start = loop[0];
    let end = loop.at(-1);
    let delta = {x: end.pos.x-start.pos.x, y: end.pos.y-start.pos.y};
    return delta;
}

function perfectLoop(cleanedLoop) {
    let netDelta = getDelta(cleanedLoop)
    let numDelt = cleanedLoop.length - 1
    let delta = {x: netDelta.x/numDelt, y: netDelta.y/numDelt};
    let closedLoop = cleanedLoop.map(({ts, pos: {x, y}}, i)=> ({ts, pos: {x: x-i*delta.x, y: y-i*delta.y}}));
    return closedLoop;
}

function rotate(space, center, amount){
    return  {
        x: Math.cos(amount) * (space.x - center.x) + Math.sin(amount) * (space.y - center.y) + center.x,
        y: Math.cos(amount) * (space.y - center.y) - Math.sin(amount) * (space.x - center.x) + center.y
    };
}
function makeHorizontal(loop) {
    let delta = getDelta(loop);
    let start = loop[0];
    let theta = Math.atan2(delta.y, delta.x);
    let flattened = loop.map(pt => {
        //theta fed here isn't negated because its calculated from delta
        let rotPt = {pos: rotate(pt.pos, start.pos, theta)};
        return Object.assign(Object.assign({}, pt), rotPt);
    })
    return flattened;
}

function scaleDim(loop, dimName, scaleFac) {
    let startPosVal = loop[0].pos[dimName];
    let scaled = loop.map(({ts, pos}) => {
        let pos2 = Object.assign({}, pos);
        let dimDelta = pos[dimName] - startPosVal;
        pos2[dimName] = startPosVal + dimDelta*scaleFac;
        return {ts, pos: pos2}
    });
    return scaled
}

let delay = ms => new Promise(resolve => setTimeout(resolve, ms))

function oscArgs(...args) {
    let output  = args.map(e => {
        let type_str = typeof e;
        switch(type_str) {
            case 'string':
                return {type: 's', value: e}
            case 'number': 
                return {type: 'f', value: e}
            case 'boolean':
                return {type: 'i', value: e+0}
        }
    });
    return output;
}

function saveLoops(fileName) {
    if(!fileName) fileName = Date().toString().replaceAll(/[\: \)\(]/g, "-") + ".json";
    fs.writeFileSync(fileName, JSON.stringify({rawLoops, processedLoops}));
}
function loadLoopFile(fileName) {
    return JSON.parse(fs.readFileSync(fileName));
}
function updateLoops() {
    const dataString = JSON.stringify({type: "processedLoops", data: processedLoops, sketchId: "runner"});
    sockets.forEach(sock => sock.write( dataString + '\n' ));
    return new Promise((resolve, reject) => setTimeout(resolve, 50));
}
function loadLoops(fileName) {
    let loops = loadLoopFile(fileName);
    rawLoops = loops.rawLoops;
    processedLoops = loops.processedLoops;
    return updateLoops();
}


//TCP server to OF

const tcp_port = 11999;
const tcp_host = '127.0.0.1';

const server = net.createServer();
server.listen(tcp_port, tcp_host, () => {
    console.log('TCP Server is running on port ' + tcp_port + '.');
});

const sockets = [];

server.on('connection', function(sock) {
    console.log('CONNECTED: ' + sock.remoteAddress + ':' + sock.remotePort);
    sockets.push(sock);

    const dataString = JSON.stringify({type: "processedLoops", data: (() => processedLoops)(), sketchId: "runner" });
    sock.write( dataString + '\n' );

    sock.on('data', function(data) {
        console.log('DATA ' + sock.remoteAddress + ': ' + data);
        // Write the data back to all the connected, the client will receive it as data from the server
        sockets.forEach(function(sock, index, array) {
            sock.write(sock.remoteAddress + ':' + sock.remotePort + " said " + data + '\n');
        });
    });

    // Add a 'close' event handler to this instance of socket
    sock.on('close', function(data) {
        let index = sockets.findIndex(function(o) {
            return o.remoteAddress === sock.remoteAddress && o.remotePort === sock.remotePort;
        })
        if (index !== -1) sockets.splice(index, 1);
        console.log('CLOSED: ' + sock.remoteAddress + ' ' + sock.remotePort);
    });
});

//OSC Server to touchOSC

const TCP_OSC_PORT = 7071;
const TCP_OSC_HOST = '192.168.7.246';

const tcp_osc_port = new osc.TCPSocketPort({});
// change to remote host/port (address of phone/tablet) - set touchOSC on device to TCPserver
tcp_osc_port.open(TCP_OSC_HOST, TCP_OSC_PORT); 

tcp_osc_port.on('ready', () => {
  console.log('ready OSC TCP');
});


const UDP_OSC_PORT = 7071;
const UDP_OSC_HOST = '0.0.0.0';

// Bind to a UDP socket to listen for incoming OSC events.
const udp_osc_port = new osc.UDPPort({
    localAddress: UDP_OSC_HOST,
    localPort: UDP_OSC_PORT,
    remoteAddress: '127.0.0.1',
    remotePort: 7072
});

udp_osc_port.on("ready", () => {
    console.log('ready OSC UDP');
});

udp_osc_port.open();

function clearAll() {
    udp_osc_port.send({address: "/clearAll", args: []});
}

let osc_port = udp_osc_port;

let rawLoops = {};
let processedLoops = {};
let activeLoop = 'firstLoop';
let isRecording = false;
let isTouching = false;
const handlers = {};

function processLoop(loopKey) {
    processedLoops[loopKey] = flipY(normalizeTime(deduplicate(sortByTs(sliceStartStop(rawLoops[loopKey])))));
}

// s_ts is server timestamp, ts is client timestamp
handlers['/recording'] = (isRec) => {
    console.log("isRec", isRec);
    isRecording = isRec;
    if(isRecording) {
        rawLoops[activeLoop] = [{s_ts: Date.now(), pos: 'start'}];
    } else {
        rawLoops[activeLoop].push({s_ts: Date.now(), pos: 'end'});
        processLoop(activeLoop);
        updateLoops();
    }
}
const f = float => ({type: 'f', value: float}); //helper for formatting floats for OSC
handlers['/pos'] = (x, y, ts, force, azimuth) => {
    if(isRecording) {
        rawLoops[activeLoop].push({s_ts: Date.now(), pos: {x, y}, ts});
        if(force != null) rawLoops[activeLoop].at(-1).force = force;
        if(azimuth != null) rawLoops[activeLoop].at(-1).azimuth = azimuth;
    }
    udp_osc_port.send({address: '/touchPos', args: [f(x), f(y)]});
    udp_osc_port.send({address: "/penForce", args: [f(force ?? 0)]});
    udp_osc_port.send({address: "/penAzimuth", args: [f(azimuth ?? 0)]});
}
handlers['/pos/touch'] = (touching) => {
    isTouching = touching;
    udp_osc_port.send({address: '/touching', args: [f(isTouching)]});
}

let logAddresses = [];
let messageHandler = (message, timetag, info) => {
    let {address, args} = message;
    if(logAddresses.includes(address) || logAddresses.includes("all")) console.log(message);
    if(handlers[address]) handlers[address](...args); 
}
osc_port.on('message', messageHandler);

let loadFile = "test_loops.json";
if(loadFile) {
    console.log("loaded file", loadFile);
    let loops = loadLoopFile(loadFile);
    processedLoops = loops.processedLoops;
    rawLoops = loops.rawLoops;
}

"Setup Done"

loaded file test_loops.json


'Setup Done'

ready OSC UDP
TCP Server is running on port 11999.
CONNECTED: 127.0.0.1:49627


Error: connect ETIMEDOUT 192.168.7.246:7071
    at TCPConnectWrap.afterConnect [as oncomplete] (node:net:1161:16)

In [9]:
logAddresses = []
handlers['/pos'] = (x, y, ts, force, azimuth) => {
    if(isRecording) {
        rawLoops[activeLoop].push({s_ts: Date.now(), pos: {x, y}, ts});
        if(force != null) rawLoops[activeLoop].at(-1).force = force;
        if(azimuth != null) rawLoops[activeLoop].at(-1).azimuth = azimuth;
    }
    udp_osc_port.send({address: '/touchPos', args: [f(x), f(y)]});
    sendOscToSketch("/penForce", force ?? 0);
    sendOscToSketch("/penAzimuth", azimuth ?? 0);
    console.log("pos", force ?? 0, azimuth ?? 0)
}

[Function (anonymous)]

pos 0.07996093481779099 0.5507538318634033
pos 0.07996093481779099 0.5522798299789429
pos 0.09259598702192307 0.5529512166976929
pos 0.020875297486782074 0.5529512166976929
pos 0.027467496693134308 0.5529512166976929
pos 0.043459683656692505 0.5529512166976929
pos 0.06402979046106339 0.5529512166976929
pos 0.08868949860334396 0.5529512166976929
pos 0.10541415214538574 0.5529512166976929
pos 0.13349203765392303 0.5550875663757324
pos 0.1541842222213745 0.5577732920646667
pos 0.18952572345733643 0.5607031583786011
pos 0.214185431599617 0.5639992952346802
pos 0.25251784920692444 0.5677836537361145
pos 0.2864554822444916 0.5719343423843384
pos 0.3103216886520386 0.5748642086982727
pos 0.326130747795105 0.5765122175216675
pos 0.3363852798938751 0.5768784880638123
pos 0.33888787031173706 0.5799304246902466
pos 0.35213330388069153 0.5824940204620361
pos 0.35567355155944824 0.5824940204620361
pos 0.3525605797767639 0.5824940204620361
pos 0.34358787536621094 0.5824940204620361
pos 0.32857230305

In [ ]:
// tight descending spirals
// udp_osc_port.send({address: "/gridSize", args: oscArgs(4)});
// udp_osc_port.send({address: "/useVoronoi", args: oscArgs(1)});
// for(var i = 0; i < 100; i++) {
//     launch = () => udp_osc_port.send({address: "/launch", args: oscArgs('loop1', 10, false, 'grp1', 'key'+i)});
//     setTimeout(launch, i*50)
// }

//wide+short descending spirals
// udp_osc_port.send({address: "/gridSize", args: oscArgs(10)});
for(var i = 0; i < 100; i++) {
    let speed = 10 + i/100;
    launch = () => udp_osc_port.send({address: "/launch", args: oscArgs('loop1', speed, false, 'grp1', 'key'+i)});
    setTimeout(launch, i*10)
}

5

In [ ]:
//sketches
async function sketch1() {
    sendOscToSketch('sketch0', "/useVoronoi", 0);
    await loadLoops('test_loops.json')
    let dev = 500; //40,
    let conf = [];
    for(var i = 0; i < dev; i++) {
        let duration = 15 + i/dev
        // udp_osc_port.send({address: "/launch", args: oscArgs('loop1', dur, false, 'grp1', 'key'+i)});
        conf.push({loopKey: 'loop1', duration})
    }
    sendConfig('sketch0', conf);
}

async function sketch2() {
    sendOscToSketch('sketch0', "/useVoronoi", 0);
    await loadLoops('test_loops.json')
    let launch = () => udp_osc_port.send({address: "/launch", args: oscArgs('loop1', 30, false, 'grp1', 'key1')});
    for(var i = 0; i < 100; i ++) {
        setTimeout(launch, i*10)
        setTimeout(launch, 4000 + i*10)
    }
}

async function sketch3() {
    sendOscToSketch('sketch0', "/gridSize", 4);
    sendOscToSketch('sketch0', "/useVoronoi", 1);
    await loadLoops('test_loops.json')
    let launch = () => udp_osc_port.send({address: "/launch", args: oscArgs('loop1', 30, false, 'grp1', 'key1')});
    for(var i = 0; i < 10; i ++) {
        setTimeout(launch, i*100)
    }
}

async function sketch4() {
    sendOscToSketch('sketch0', "/gridSize", 4);
    sendOscToSketch('sketch0', "/useVoronoi", 1);
    await loadLoops('test_loops.json')
    let launch = () => udp_osc_port.send({address: "/launch", args: oscArgs('loop1', 60, false, 'grp1', 'key1')});
    for(var i = 0; i < 10; i ++) {
        setTimeout(launch, i*100)
        setTimeout(launch, 4000 + i*100)
    }
}

async function sketch5() {
    sendOscToSketch('sketch0', "/gridSize", 4);
    sendOscToSketch('sketch0', "/useVoronoi", 1);
    await loadLoops('test_loops.json')
    processedLoops['loop2_flat'] = makeHorizontal(processedLoops['loop2'])
    let xDelt = getDelta(processedLoops['loop2_flat']).x
    processedLoops['loop2_flat_unit'] = scaleDim(processedLoops['loop2_flat'], 'x', 1/xDelt)
    await updateLoops()
    let conf = [];
    let num = 8
    for(var i = 0; i < num; i++) {
        conf.push({loopKey: 'loop2_flat_unit', rotation: i/num * Math.PI * 2, pos: {x: 0.5, y: 0.5}})
    }
    sendConfig('sketch0', conf)
    conf.forEach(c => {c.duration = 3});
    await delay(300)
    sendConfig('sketch0', conf)
}

async function sketch6() {
    sendOscToSketch('sketch0', "/gridSize", 10);
    sendOscToSketch('sketch0', "/useVoronoi", 1);
    await loadLoops('test_loops.json')
    processedLoops['loop2_flat'] = makeHorizontal(processedLoops['loop2'])
    let xDelt = getDelta(processedLoops['loop2_flat']).x
    processedLoops['loop2_flat_unit'] = scaleDim(processedLoops['loop2_flat'], 'x', 1/xDelt)
    await updateLoops()
    let conf = [];
    let num = 8
    for(var i = 0; i < num; i++) {
        conf.push({loopKey: 'loop2_flat_unit', rotation: i/num * Math.PI * 2, pos: {x: 0.5, y: 0.5}, looping: false})
    }
    sendConfig('sketch0', conf)
    conf.forEach(c => {c.duration = 1});
    for(var i = 0; i < 5; i ++) {
        await delay(700)
        sendConfig('sketch0', conf)
    }
}

async function sketch7() {
    //NOTE - need to remove ofDisableAlphaBlending(); in ofApp_1.cpp::setup() for this to look like original
    sendOscToSketch('sketch1', "/schemeInd", 1); //scheme 2 is interesting with ofDisableAlphaBlending() called, play with "/delayTime" param 
    await loadLoops('test_loops.json')
    processedLoops['loop2_flat'] = makeHorizontal(processedLoops['loop2'])
    let xDelt = getDelta(processedLoops['loop2_flat']).x
    processedLoops['loop2_flat_unit'] = scaleDim(processedLoops['loop2_flat'], 'x', 1/xDelt)
    await updateLoops()
    let conf = [];
    let num = 8
    for(var i = 0; i < num; i++) {
        conf.push({loopKey: 'loop2_flat_unit', rotation: i/num * Math.PI * 2, pos: {x: 0.5, y: 0.5}})
    }
    sendConfig('sketch1', conf)
    conf.forEach(c => {c.duration = 3});
    await delay(300)
    sendConfig('sketch1', conf)
}

In [ ]:
// clearSketch('sketch1')
sketch7()

In [ ]:
sendOscToSketch('sketch1', "/schemeInd", 3);
sendOscToSketch('sketch1', "/delayTime", 5);
start = 0.2417727293033587
dur = .00045
sendOscToSketch('sketch1', "/loopPosition", start, start+dur);
[0.17968565414851656, 0.20409988857550232, 0.5106165883448111, 0.08039998895633138, 0.2417727293033587]

In [ ]:
async function loopCuts(){
    durBase = 0.00025/ 4
    mults = [1, 1, 2, 2, 4, 8] 
    // starts = [1,2,3,4,5].map(e => Math.random())
    starts = [0.17968565414851656, 0.20409988857550232, 0.5106165883448111, 0.08039998895633138, 0.2417727293033587]
    console.log(starts, mults)
    select = arr => arr[Math.floor(Math.random() * arr.length)];
    outs = [];
    for(var i = 0; i < 40; i++) {
        start = select(starts)
        dur = durBase * select(mults)
        sendOscToSketch('sketch1', "/loopPosition", start, start+dur);
        outs.push([start, dur])
        await delay(1000)
    }
    start = Math.random()
    dur = .025
    sendOscToSketch('sketch1', "/loopPosition", start, start+dur);
    console.log(outs)
}

loopCuts()

In [ ]:
//basic ws template

const ws = require('ws')

const wss = new ws.Server({ port: 8080 });

wss.on('connection', function connection(ws) {
  console.log("connected")
  ws.on('message', function message(data) {
    console.log('received: %s', data);
  });
});

In [ ]:
190.52